### Esta columna me va a volver loca

In [19]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
import statsmodels.api as sm

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
anime = pd.read_csv('Anime_raw.csv', sep=",")
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21460 entries, 0 to 21459
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                21460 non-null  int64  
 1   Title             21460 non-null  object 
 2   Synonyms          21460 non-null  object 
 3   Japanese          21460 non-null  object 
 4   English           21460 non-null  object 
 5   Synopsis          21460 non-null  object 
 6   Type              21460 non-null  object 
 7   Episodes          20913 non-null  float64
 8   Status            21460 non-null  object 
 9   Start_Aired       21460 non-null  object 
 10  End_Aired         21460 non-null  object 
 11  Premiered         21460 non-null  object 
 12  Broadcast         21460 non-null  object 
 13  Producers         21460 non-null  object 
 14  Licensors         21460 non-null  object 
 15  Studios           21460 non-null  object 
 16  Source            21460 non-null  object

In [22]:
#La columna start_aired es object. Necesito que sea int y que solo aparezca el año. 
anime["Start_Aired"].value_counts()

Unknown         374
2010             89
2011             85
2005             84
2012             75
               ... 
Nov 22, 2002      1
Oct 9, 1981       1
Feb 10, 2004      1
Nov 11, 1966      1
May 12, 2018      1
Name: Start_Aired, Length: 8221, dtype: int64

In [23]:
#Idea: quitar Unknowns y los sustituimos por el outlier 1900
#Máscara para que me muestre los unknowns. Hay 374
mask=(anime["Start_Aired"] == 'Unknown')
anime["Start_Aired"].loc[mask]

833      Unknown
1382     Unknown
1537     Unknown
1703     Unknown
1805     Unknown
          ...   
21433    Unknown
21446    Unknown
21447    Unknown
21449    Unknown
21451    Unknown
Name: Start_Aired, Length: 374, dtype: object

In [24]:
#Ahora que me muestre si hay algún anime en 1900 de base. Serie vacía. No hay.
mask=(anime["Start_Aired"] == "1900")
anime["Start_Aired"].loc[mask]

Series([], Name: Start_Aired, dtype: object)

In [25]:
#Por cada elemento de la columna start_aired que sea Unknown, lo convierto en 1900 como número
for i in anime["Start_Aired"]:
    if i == 'Unknown':
        anime["Start_Aired"][anime["Start_Aired"] == 'Unknown'] = 1900

In [26]:
#Vuelvo a verificar si ahora hay 1900 ( en número). Hay 374, los mismos que antes eran Unk.
mask=(anime["Start_Aired"] == 1900)
anime["Start_Aired"].loc[mask]

833      1900
1382     1900
1537     1900
1703     1900
1805     1900
         ... 
21433    1900
21446    1900
21447    1900
21449    1900
21451    1900
Name: Start_Aired, Length: 374, dtype: object

In [27]:
anime["Start_Aired"].value_counts()

1900            374
2010             89
2011             85
2005             84
2012             75
               ... 
Nov 22, 2002      1
Oct 9, 1981       1
Feb 10, 2004      1
Nov 11, 1966      1
May 12, 2018      1
Name: Start_Aired, Length: 8221, dtype: int64

In [29]:
#Vamos a convertir las fechas con formato "May 12, 2018" en 2018 (ejemplo). Buscamos una fila para hacer la prueba. 
anime["Start_Aired"].iloc[0]

'Apr 7, 2013'

In [30]:
#La primera fila nos vale. Vamos a hacer un split y que nos deje solo el año en formato int
int(anime["Start_Aired"].iloc[0].split()[-1])

2013

In [31]:
#Vamos a crear un bucle para que recorra toda la columna y convierta a int las fechas con formato "May 12, 2018", dejando solo el año. 
errores = pd.DataFrame()
for pos in range(len(anime["Start_Aired"])):
    try:
        if type(anime["Start_Aired"].iloc[pos]) == str:
            # anime["Start_Aired"][pos]
            anime["Start_Aired"].iloc[pos]=int(anime["Start_Aired"].iloc[pos].split()[-1])
    except:
        x=anime["Start_Aired"].iloc[pos]
        errores.append([pos,x])

In [32]:
#Parece que ya no hay errores
errores

""


In [33]:
#Hasta aquí bien, ya tenemos una gran cantidad de fechas con formato "solo año y string". Ahora aparece un nuevo formato de fecha: Jan-16
anime["Start_Aired"].value_counts()

2017      1068
2018      1044
2016      1023
2019       958
2014       949
          ... 
Jan-68       1
May-13       1
Mar-12       1
Jun-03       1
Jan-16       1
Name: Start_Aired, Length: 191, dtype: int64

In [34]:
#Partiendo de la base de que el anime comenzó el 1917 con un cortometraje que no está en la base de datos, podemos decir sin equivocarnos
#que todo lo que sea menor de 24 será de los años 2000. 
# Creamos un bucle que nos busque todas las fechas que siguen siendo string, haga un split cuando encuentre un guión y,
#si el número tras el guión es menos o igual a 24, le sumamos 2000. En caso contrario, sumamos 1900.
errores = pd.DataFrame()
for pos in range(len(anime["Start_Aired"])):
    try:
        if type(anime["Start_Aired"].iloc[pos]) == str:
            aniored=int(anime["Start_Aired"].iloc[pos].split("-")[-1])
            if aniored<=24:
                anio=2000+aniored
            else:
                anio=1900+aniored          
            anime["Start_Aired"].iloc[pos]=anio
    except:
        x=anime["Start_Aired"].iloc[pos]
        errores.append([pos,x])


In [35]:
#Ya solo aparecen años, pero los que originalmente tenían el formato que queríamos siguen siendo string.
anime["Start_Aired"]

0        2013
1        2006
2        2009
3        2015
4        2012
         ... 
21455    2018
21456    2020
21457    2018
21458    2018
21459    2018
Name: Start_Aired, Length: 21460, dtype: object

In [40]:
#Cambiamos el tipo de los datos de la columna a int
anime["Start_Aired"] = anime["Start_Aired"].astype(int)

In [41]:
#Verificamos si funcionó. Start_Aired -->21460 non-null -> int32 (Funcionó!)
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21460 entries, 0 to 21459
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                21460 non-null  int64  
 1   Title             21460 non-null  object 
 2   Synonyms          21460 non-null  object 
 3   Japanese          21460 non-null  object 
 4   English           21460 non-null  object 
 5   Synopsis          21460 non-null  object 
 6   Type              21460 non-null  object 
 7   Episodes          20913 non-null  float64
 8   Status            21460 non-null  object 
 9   Start_Aired       21460 non-null  int32  
 10  End_Aired         21460 non-null  object 
 11  Premiered         21460 non-null  object 
 12  Broadcast         21460 non-null  object 
 13  Producers         21460 non-null  object 
 14  Licensors         21460 non-null  object 
 15  Studios           21460 non-null  object 
 16  Source            21460 non-null  object